# Combining Python, SQL DBs and AWS

## Aim: Using python, create a DB on your AWS server, insert data into it and query that data to answer questions. 


# Using Python with MYSQL DB

In [1]:
# make sure we have the package installed
!pip install mysql-connector-python

In [2]:
## importing 'mysql.connector' 
import mysql.connector 
import config1

In [3]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = config1.host,
    user = config1.user,
    passwd = config1.password
)

### Documentation:

https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html

## Using a config File:

You do not want to make your credentials viewable to everyone who might see this file.  So instead of explicity stating your credentials, we want to import them from another file.  

In [202]:
import config

ModuleNotFoundError: No module named 'config'

In [203]:
config.pw_example

NameError: name 'config' is not defined

In [4]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host = config1.host,
    user = config1.user,
    passwd = config1.password
)

print(cnx) # it will print a connection object if everything is fine


In [5]:
cursor = cnx.cursor()

## Creating DB

In [6]:
import mysql.connector
from mysql.connector import errorcode

In [7]:
db_name = 'students'

Function to help us create a database

In [8]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [9]:
# check to see if the database already 
try:
    cursor.execute("USE {}".format(db_name))

#if the previous line fails because there isn't a db by that name run this line

except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

### Creating tables

In [10]:
DB_NAME = 'employees'

TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE employees ("
    "  emp_no int(11) NOT NULL AUTO_INCREMENT,"
    "  birth_date date NOT NULL,"
    "  first_name varchar(14) NOT NULL,"
    "  last_name varchar(16) NOT NULL,"
    "  gender enum('M','F') NOT NULL,"
    "  hire_date date NOT NULL,"
    "  PRIMARY KEY (emp_no)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE departments ("
    "  dept_no char(4) NOT NULL,"
    "  dept_name varchar(40) NOT NULL,"
    "  PRIMARY KEY (dept_no), UNIQUE KEY dept_name (dept_name)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE salaries ("
    "  emp_no int(11) NOT NULL,"
    "  salary int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE dept_emp ("
    "  emp_no int(11) NOT NULL,"
    "  dept_no char(4) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE dept_manager ("
    "  dept_no char(4) NOT NULL,"
    "  emp_no int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no),"
    "  KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_manager_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_manager_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE titles ("
    "  emp_no int(11) NOT NULL,"
    "  title varchar(50) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date DEFAULT NULL,"
    "  PRIMARY KEY (emp_no,title,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT titles_ibfk_1 FOREIGN KEY (emp_no)"
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [11]:
TABLES['employees']

"CREATE TABLE employees (  emp_no int(11) NOT NULL AUTO_INCREMENT,  birth_date date NOT NULL,  first_name varchar(14) NOT NULL,  last_name varchar(16) NOT NULL,  gender enum('M','F') NOT NULL,  hire_date date NOT NULL,  PRIMARY KEY (emp_no)) ENGINE=InnoDB"

In [218]:
cnx = mysql.connector.connect(
    host = config1.host,
    user = config1.user,
    passwd = config1.password,
    database = DB_NAME
)
cursor = cnx.cursor()

ProgrammingError: 1049 (42000): Unknown database 'employees'

In [219]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table employees: already exists.
Creating table departments: already exists.
Creating table salaries: already exists.
Creating table dept_emp: already exists.
Creating table dept_manager: already exists.
Creating table titles: already exists.


## Inserting Data

In [224]:
from datetime import date, datetime, timedelta

#since we closed the connection, we need to reinstatiate it
cnx = mysql.connector.connect(
    host = config1.host,
    user = config1.user,
    passwd = config1.password,
    database = db_name
)
cursor = cnx.cursor()

In [225]:
tomorrow = datetime.now().date() + timedelta(days=1)

In [226]:
# insert statement to add an employee
add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")

#insert statement to add a salary
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

#data to insert for one employee
data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))



In [227]:
# Insert new employee
cursor.execute(add_employee, data_employee)
# Make sure data is committed to the database
cnx.commit()

In [228]:
# grabs the id of the employee we just inserted so we can use it to input salary data
emp_no = cursor.lastrowid

In [229]:

# create salary information
data_salary = {
  'emp_no': emp_no,
  'salary': 50000,
  'from_date': tomorrow,
  'to_date': date(9999, 1, 1),
}

In [230]:

#Insert that salary information
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

## Insert Many

If you have a list of records that you would like to input, there are two ways you can do this.  YOu can loop over each record in python and execute each record, or you can use the `executemany()` method.  

`cursor.executemany(operation, list_of_params)`



In [231]:
#create a list of records

data = [
  ('Jane', date(2005, 2, 12)),
  ('Joe', date(2006, 5, 23)),
  ('John', date(2010, 10, 3)),
]

In [232]:
#create the insert statment
stmt = "INSERT INTO employees (first_name, hire_date) VALUES (%s, %s)"


In [233]:
#insert all of the records and commit it
cursor.executemany(stmt, data)
cnx.commit()

In [234]:
#Close the connection 

cursor.close()
cnx.close()

## Quyerying the DB

In [236]:
cnx = mysql.connector.connect(
    host = config1.host,
    user = config1.user,
    passwd = config1.password,
    database = DB_NAME
)
cursor = cnx.cursor()

ProgrammingError: 1049 (42000): Unknown database 'employees'

In [240]:

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = datetime.date(2006, 1, 1)
hire_end = datetime.date(2006, 12, 31)

cursor.execute(query, (hire_start, hire_end))



TypeError: descriptor 'date' requires a 'datetime.datetime' object but received a 'int'

### How to read the data from your query.

You can use the `fetchone()`, `fetchmany()`, or `fetchall()` methods.
 


The `fetchone()` method returns the next row of a query result set or `None` in case there is no row left.

        row = cursor.fetchone()
 
        while row is not None:
            print(row)
            row = cursor.fetchone()

In case the number of rows in the table is small, you can use the `fetchall()` method to fetch all rows from the database table.

        rows = cursor.fetchall()
 
        print('Total Row(s):', cursor.rowcount)
        for row in rows:
            print(row)

**Why might you use a fetchone() when you have a query that will return multiple rows?**

For a relatively big table, it takes time to fetch all rows and return the entire result set. In addition, `fetchall()` needs to allocate enough memory to store the entire result set in the memory, which is not efficient.

MySQL Connector/Python has the `fetchmany()` method that returns the next number of rows (n) of the result set, which allows you to balance between retrieval time and memory space.



First, develop a generator that chunks the database calls into a series of `fetchmany()` calls:

In [239]:
def iter_row(cursor, size=10):
    while True:
        rows = cursor.fetchmany(size)
        if not rows:
            break
        for row in rows:
            yield row

Second, use the  iter_row() generator to fetch 10 rows at a time :

`cursor.execute("SELECT * FROM books")`
 
`for row in iter_row(cursor, 10):
    print(row)`

In [238]:
#in practice

data = cursor.fetchall()

AttributeError: 'NoneType' object has no attribute 'unread_result'

In [ ]:
print(data)

In [ ]:
cursor.close()
cnx.close()

## Applied: Working with our student data

### Import our student data

In [17]:
import json

f=open('students.json','r')
data=json.load(f)

In [18]:
#examine that student data
len(data)
data[0].keys()

dict_keys(['name', 'dob', 'fav_food', 'birthplace', 'years_in_nyc'])

### Determine how you should define your table to hold the data



### Create a connection and cursor to AWS db

In [19]:
import config1
cnx = mysql.connector.connect(
    host = config1.host,
    user = config1.user,
    passwd = config1.password
)

cursor = cnx.cursor()

In [30]:
cursor.execute("CREATE DATABASE students")

DatabaseError: 1007 (HY000): Can't create database 'students'; database exists

In [31]:
cnx.database = 'students'

### Create a table for our student info

In [32]:

create_query = """CREATE TABLE students
    (name varchar(100),
    dob date,
    fav_food varchar(100),
    birthplace varchar (100),
    years_in_nyc float
    );
    """
# we created a table called students, assigned to the variable, create_query.

In [33]:
cursor.execute(create_query)
# we then executed this query, and created a table with categories name, dob, fav food, birthplace, years in nyc
# with the types of values that they should be associated with.

ProgrammingError: 1050 (42S01): Table 'students' already exists

### Now that we have created our table we need to insert our data into the table.

We can't just pass the execute query a list of dictioanries,  We need to parse through it first. 

Take one student and identify how you would parse that information into a usable format.

In [34]:
student = data[0]
student
# data is the file that we imported, with all the student data.
# we assign the first data line to the variable, student.


{'name': 'Anastasia Gorina',
 'dob': '1991-07-19',
 'fav_food': 'oysters',
 'birthplace': 'St. Petersburg, Russia',
 'years_in_nyc': 5}

Now use a for loop to loop over all of data and return a list that we can use for `executemany()`.

**If all of your data isn't formatted as expected, this will cause your loop to fail**  you might have to account for 'bad data' in your for loop.

In [35]:
(student['name'], student['birthplace'], student['dob'], student['fav_food'], student['years_in_nyc'])

('Anastasia Gorina', 'St. Petersburg, Russia', '1991-07-19', 'oysters', 5)

In [36]:
students_list = []
    
for student in data:
    if 'birthdate' in student:
        student['dob'] = student['birthdate']

    student_tuple = (student['birthplace'], student['dob'], student['fav_food'], student['name'], student['years_in_nyc'])
    students_list.append(student_tuple)

# we looped through and made the dictionary into a tuple, then appended it into the students_list.
# students_list now has only values in it, without the category names.
students_list

[('St. Petersburg, Russia', '1991-07-19', 'oysters', 'Anastasia Gorina', 5),
 ('Bogota, CU', '1983-01-22', 'chocolate', 'Andres M Chaves', 6),
 ('Washington DC', '1989-10-17', 'baked ziti', 'Andrew Triola', 3),
 ('Fuzhou, China', '1990-12-17', 'Seafood Boil', 'Anita Guo', 16),
 ('Bishkek, Kyrgyzstan', '1981-25-02', 'cheese cake', 'Askhat Yktybaev', 3),
 ('Bronx, NY', '1994-04-29', 'Burgers', 'Aviva Sally Mazurek', 2),
 ('London, UK',
  '1900-04-01',
  'Roast Beef and Yorkshire pudding',
  'Ben Johnson-Laird',
  21),
 ('Painesville, OH', '1992-05-27', 'noodles', 'Brittany Fowle', 9),
 ('Queens, New York', '1991-07-29', 'chicken', 'Chadwick Balloo', 1),
 ('Glen Ridge, NJ', '1995-02-24', 'pizza', 'Ed Haracz', 0),
 ('Flushing, NY', '1990-04-05', 'sushi', 'Grace Park', 29),
 ('Nashville, TN', '1987-11-28', 'fried_chicken', 'Jackson Bull', 8),
 ('Portland, OR', '1996-11-27', 'soup dumplings', 'Julia Chong', 1.5),
 ('OKC, OK', '1996-05-04', 'Burgers', 'Owen Peng', 13),
 ('New York City', '197

Now that we have this list of tuples, lets use `executemany()` to insert the data.

In [37]:
stmt = "INSERT INTO students (birthplace, dob, fav_food, name, in_nyc) VALUES (%s, %s, %s, %s, %s)"
# we go back into our students table, and insert category names and values.

In [38]:
students_list[0]

('St. Petersburg, Russia', '1991-07-19', 'oysters', 'Anastasia Gorina', 5)

In [39]:
cursor.execute(stmt, students_list[0])

In [40]:
cursor.executemany(stmt, students_list)
cnx.commit()

Now that we have the data in the database, we can write

### Write queries to answer the following questions:

**Questions**
- Which student was born closest to the cohort's graduation date?
- Which student has the most siblings?
- How many students are only children?
- Which 3 students have lived in NYC the shortest amount of time?
- How many students are native New Yorkers?
- Do any two students have the same favorite food?

In [319]:
cursor.execute("SELECT name FROM students")

In [322]:
cursor.execute("SELECT name, count(name) FROM students")
a = cursor.fetchall()


ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'count(name) FROM students' at line 1